In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Conexión (ajusta si el notebook está dentro de /scripts)
conn = sqlite3.connect("../database/dodo_supermercado.db")

# Consulta SQL para obtener datos de ventas y supermercados, desde 2023 al 2024
# Conocer la demanda en diferentes ciudades y categorías de productos para análisis de series temporales

# Consulta SQL con CTE y rango de fechas (2023-2024)
# CTE = Common Table Expression/Expresión de Tabla Común
query = """
WITH base AS (
    SELECT v.Fecha, v.Categoria, v.Cantidad_Vendida, s.Ciudad, s.Nombre_Supermercado, v.Id_Tienda
    FROM Ventas v
    JOIN Supermercado s ON v.Id_Tienda = s.Id_Tienda
    WHERE v.Fecha >= '2023-01-01' AND v.Fecha < '2025-01-01'
    )
SELECT * FROM base
ORDER BY Ciudad, Fecha;
"""

# Carga de datos al DataFrame
df = pd.read_sql(query, conn, parse_dates=["Fecha"])
conn.close()

# Procesamiento de fechas para análisis temporal
df["Año"] = df["Fecha"].dt.year
df["Mes"] = df["Fecha"].dt.to_period("M")

# Verificación de datos cargados
print("Datos cargados")
print(df.head(6))
print(f"\n Rango de fechas: {df['Fecha'].min()} a {df['Fecha'].max()}")
print(f"Ciudades analizadas: {df['Ciudad'].unique()}")

In [ ]:
# Agrupar ventas por Ciudad y Categoria
demanda_total = (
    df.groupby(["Ciudad", "Categoria"])["Cantidad_Vendida"].sum().reset_index()
)

# CATEGORÍA MÁS VENDIDA POR CIUDAD
top_categorias = (
    demanda_total.sort_values(["Ciudad", "Cantidad_Vendida"], ascending=[True, False])
    .groupby("Ciudad")
    .head(1)
)

print("=== CATEGORÍA MÁS VENDIDA POR CIUDAD ===")
print(top_categorias)

# CATEGORÍA MENOS VENDIDA POR CIUDAD
bottom_categorias = (
    demanda_total.sort_values(["Ciudad", "Cantidad_Vendida"], ascending=[True, True])
    .groupby("Ciudad")
    .head(1)
)

print("\n=== CATEGORÍA MENOS VENDIDA POR CIUDAD ===")
print(bottom_categorias)

In [ ]:
# Top Ciudad por mayor ventas de Categoria a Nivel General/Nacional
# Mostrar la ciudad con mayor demanda total de todas las categorías
top_ciudad = demanda_total.sort_values("Cantidad_Vendida", ascending=False).head(1)
print("Ciudad con mayor demanda")
print(top_ciudad)

# Top ciudad con menor demanda de Categoria a Nivel General/Nacional
# Mostrar la ciudad con menor demanda total de todas las categorías
print("\n********************************************** \n")
menor_demanda_ciudad = demanda_total.sort_values(
    "Cantidad_Vendida", ascending=True
).head(1)
print("Ciudad con menor demanda")
print(menor_demanda_ciudad)

In [ ]:
# Demanda mensual por ciudad y categoría
demanda_mensual = (
    df.groupby(["Ciudad", "Categoria", "Mes"])["Cantidad_Vendida"].sum().reset_index()
)

# Visualización de la demanda mensual para la categoría más demandada en cada ciudad
ciudades = df["Ciudad"].unique()

# Ciclo para graficar la demanda mensual por ciudad
# Usamos un gráfico de líneas para cada ciudad
for ciudad in ciudades:
    data = demanda_mensual[demanda_mensual["Ciudad"] == ciudad]
    pivot = data.pivot(
        index="Mes", columns="Categoria", values="Cantidad_Vendida"
    ).fillna(0)
    pivot.plot(figsize=(10, 6), title=f"Demanda Mensual en {ciudad} (2023-2024)")
    plt.xlabel("Mes")
    plt.ylabel("Cantidad Vendida")
    plt.legend(title="Categoría")
    plt.grid(alpha=0.3)
    # Guardar las figuras en "outputs" y que en caso de existir sobreescriba así no generamos múltiples archivos
    plt.savefig(f"../outputs/demanda_mensual_{ciudad.replace(' ', '_')}.png")

In [ ]:
# Ranking general de categorías por cantidad vendida, suma de todas las ciudades
ranking = (
    df.groupby(["Categoria"])["Cantidad_Vendida"]
    .sum()
    .reset_index()
    .sort_values("Cantidad_Vendida", ascending=False)
)
print("Ranking general de categorías por cantidad vendida (2023-2024):")
print(ranking)

# Visualización del ranking general de categorías
plt.figure(figsize=(18, 8))
plt.bar(ranking["Categoria"], ranking["Cantidad_Vendida"], color="skyblue")
plt.title("Ranking General de Categorías por Cantidad Vendida (2023-2024)")
plt.xlabel("Categoría")
plt.ylabel("Cantidad Vendida")
plt.xticks(rotation=45)
plt.grid(axis="y", alpha=0.3)
# Se guarda la figura en "outputs" y en caso de existir sobreescriba así no generamos múltiples archivos
plt.savefig("../outputs/ranking_general_categorias.png")

In [ ]:
# Definir las ciudades y categorías
casos = [("arica", "bebidas"), ("antofagasta", "congelados")]

for ciudad, categoria in casos:
    sql = """
    SELECT v.Fecha, SUM(v.Cantidad_Vendida) AS Cantidad_Vendida
    FROM Ventas v
    JOIN Supermercado s ON v.Id_Tienda = s.Id_Tienda
    WHERE (s.Ciudad) = ?
      AND (v.Categoria) = ?
      AND v.Fecha BETWEEN '2023-01-01' AND '2024-12-31'
    GROUP BY v.Fecha
    ORDER BY v.Fecha;
    """
    with sqlite3.connect("../database/dodo_supermercado.db") as conn:
        df_diario = pd.read_sql_query(
            sql, conn, params=(ciudad, categoria), parse_dates=["Fecha"]
        )

    print(f"\n=== {ciudad.upper()} · {categoria.upper()} ===")
    print(df_diario.head(5))
    print(
        f"Total registros: {len(df_diario)}  |  Desde {df_diario['Fecha'].min().date()} hasta {df_diario['Fecha'].max().date()}"
    )

In [ ]:
promedio_diario = df_diario["Cantidad_Vendida"].mean()
print(f"\n=== {ciudad.upper()} · {categoria.upper()} ===")
print(f"Promedio diario de ventas: {promedio_diario:.0f}")

In [ ]:
df_diario["Mes"] = df_diario["Fecha"].dt.to_period("M")
promedio_mensual = (
    df_diario.groupby("Mes")["Cantidad_Vendida"].mean().reset_index().sort_values("Mes")
)
print(promedio_mensual.head(24))

In [ ]:
for ciudad, categoria in [("arica", "bebidas"), ("antofagasta", "congelados")]:
    sql = """
    SELECT v.Fecha, SUM(v.Cantidad_Vendida) AS Cantidad_Vendida
    FROM Ventas v
    JOIN Supermercado s ON v.Id_Tienda = s.Id_Tienda
    WHERE (s.Ciudad) = ?
      AND (v.Categoria) = ?
      AND v.Fecha BETWEEN '2023-01-01' AND '2024-12-31'
    GROUP BY v.Fecha
    ORDER BY v.Fecha;
    """

    #  Aquí estaba el problema: faltaba volver a leer df_diario en cada iteración
    with sqlite3.connect("../database/dodo_supermercado.db") as conn:
        df_diario = pd.read_sql_query(
            sql, conn, params=(ciudad, categoria), parse_dates=["Fecha"]
        )

    promedio_diario = df_diario["Cantidad_Vendida"].mean()
    variabilidad = df_diario["Cantidad_Vendida"].std()
    coef_var = (variabilidad / promedio_diario) * 100

    print(f"\n=== {ciudad.upper()} · {categoria.upper()} ===")
    print(f"Promedio diario de ventas: {promedio_diario:.2f}")
    print(f"Desviación estándar diaria: {variabilidad:.2f}")
    print(f"Coeficiente de variación: {coef_var:.2f}%")